# Type et méthodes

JULIA possède un système de type et de méthode qui lui confère une approche objet.
La fonction typeof() renvoie le type d'une variable de base Int32, Float64... JULIA est conçu pour permettre facilement d'étendre l'environnement à de nouveaux type de variable.

Le types sont organisés suivant un hiérarchie comme on peut le voir sur l'arborescence partielle ci-dessous

(arborescence générée à l'aide de https://github.com/tanmaykm/julia_types/blob/master/julia_types.jl)

In [8]:
function f(x::Any)
    gamma(x+1)
end

f (generic function with 2 methods)

In [9]:
function f(n::Integer)
    factorial(n)
end

f (generic function with 2 methods)

In [10]:
f(3.0)

6.0

In [11]:
f(3)

6

In [12]:
f(im)

0.4980156681183314 - 0.15494982830177254im

In [13]:
f(-2)

LoadError: DomainError
while loading In[13], in expression starting on line 1

In [14]:
factorial(sqrt(2))

LoadError: `factorial` has no method matching factorial(::Float64)
while loading In[14], in expression starting on line 1

In [84]:
abstract Grid # juste en dessous de Any
type Grid1d <: Grid
    debut::Float64
    fin::Float64
    n::Int32
end

In [85]:
a=Grid1d(0,1,2)

LoadError: InexactError()
while loading In[85], in expression starting on line 1

In [19]:
a.debut

0.0

In [20]:
a.fin

1.0

In [21]:
a.n

2

In [37]:
A=speye(3,3)
A[1,2]=2

2

In [38]:
typeof(A)

SparseMatrixCSC{Float64,Int64} (constructor with 1 method)

In [39]:
A.rowval

4-element Array{Int64,1}:
 1
 1
 2
 3

In [40]:
A.colptr

4-element Array{Int64,1}:
 1
 2
 4
 5

In [41]:
A.nzval

4-element Array{Float64,1}:
 1.0
 2.0
 1.0
 1.0

In [33]:
A.m

3

In [34]:
A.n

3

In [42]:
+

+ (generic function with 119 methods)

In [86]:
function +(g::Grid1d,n::Integer)
    g.n+=n
    return g
end

+ (generic function with 120 methods)

In [87]:
a=Grid1d(0,1,2)

Grid 1d : début 0.0 , fin 1.0 , 2 éléments


In [88]:
a+2

Grid 1d : début 0.0 , fin 1.0 , 4 éléments


In [90]:
a

Grid 1d : début 0.0 , fin 1.0 , 4 éléments


In [92]:
2+a

LoadError: `+` has no method matching +(::Int64, ::Grid1d)
while loading In[92], in expression starting on line 1

In [95]:
+a

LoadError: `+` has no method matching +(::Grid1d)
while loading In[95], in expression starting on line 1

In [96]:
a+[1,2]

LoadError: `+` has no method matching +(::Grid1d, ::Array{Int64,1})
while loading In[96], in expression starting on line 1

In [56]:
println(a)

Grid1d(0.0,1.0,4)


In [57]:
Base.show(a)

Grid1d(0.0,1.0,4)

In [73]:
function Base.show(io::IO,g::Grid1d)
    print(io, "Grid 1d : début $(g.debut) , fin $(g.fin) , $(g.n) éléments\n")
end

show (generic function with 92 methods)

In [74]:
a

Grid 1d : début 0.0 , fin 1.0 , 4 éléments


In [76]:
function Base.size(g::Grid)
    return g.n
end

size (generic function with 52 methods)

In [77]:
size(a)

4

In [78]:
abstract Grid # juste en dessous de Any
type Grid1d <: Grid
    debut::Float64
    fin::Float64
    n::Int32
    
    # constructeurs par défaut sans argument
    function Grid1d()
        new(0,0,0)
    end
    
    # constructeurs par défaut avec argument
    function Grid1d(a,b,c)
        if c<=0
            error("pas possible")
        else
            new(a,b,c)
        end
    end
end

In [82]:
b=Grid1d(0,1,-1)

LoadError: pas possible
while loading In[82], in expression starting on line 1

In [97]:
function Base.det(g::Grid1d)
    

LoadError: `det` has no method matching det(::Grid1d)
while loading In[97], in expression starting on line 1

In [98]:
a

Grid 1d : début 0.0 , fin 1.0 , 4 éléments


In [99]:
a[1]

LoadError: `getindex` has no method matching getindex(::Grid1d, ::Int64)
while loading In[99], in expression starting on line 1

LoadError: type Grid1d has no field h
while loading In[100], in expression starting on line 1